In [1]:
import pandas

In [2]:
lines = None
with open("data/features", 'r') as stream:
    lines = map(lambda x: x.strip(), stream.readlines())

In [3]:
repro = pandas.read_csv("data/repro.csv")

In [4]:
table = dict()
for elt in lines: 
    table[elt] = [None for _ in range(1000)]
table["BUILD?"] = list(repro["build?"])
table["REPRO?"] = list(repro["reproducible?"])

In [5]:
for i in range(1, 1001):
    config = f"configs/{i:04d}_randconfig"
    with open(config, 'r') as stream:
        for line in stream:
            if line.startswith('#'):
                continue
            if line.startswith("CONFIG_"):
                name, value = line.strip().split('=')
                name = name[7:]
                table[name][i-1] = value

In [7]:
df = pandas.DataFrame.from_dict(table)

In [8]:
df

,060_WRITETHROUGH,104_QUAD_8,16KSTACKS,21285_WATCHDOG,29BIT,32BIT,3C515,405EX,405EZ,40x,...,ZSWAP_ZPOOL_DEFAULT_Z3FOLD,ZSWAP_ZPOOL_DEFAULT_ZBUD,ZSWAP_ZPOOL_DEFAULT_ZSMALLOC,ZYNQMP_FIRMWARE,ZYNQMP_FIRMWARE_DEBUG,ZYNQMP_IPI_MBOX,ZYNQMP_PM_DOMAINS,ZYNQMP_POWER,BUILD?,REPRO?
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,False
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,False
996,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
997,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
998,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,True,True
